# Introduction

This notebook deals with finding effective, or pseudo-effective builds

In [1]:
%load_ext autoreload
%autoreload 2

from pymongo import MongoClient
from bson.objectid import ObjectId

import analysis_utilities

mongo_client = MongoClient()

database = mongo_client["graduation"]

pull_requests_collection = database["pull_requests"]

projects_collection = database["projects"]

commits_collection = database["commits"]

projects = list(projects_collection.find({'succeeded' : True, 'travis_is_oldest_ci': True}))

print("Found {} projects with a travis build status date".format(len(projects)))

Found 685 projects with a travis build status date


In [ ]:
for project in projects:
    pull_requests = list(pull_requests_collection.find({
        'project_name': project["full_name"].split("/")[1],
        'project_owner': project["full_name"].split("/")[0],
    }))
    
    after = analysis_utilities.split_prs_on_build_date(project, pull_requests, field_name= "status_travis_date", days_delta = 0)[1]
    
    for pr in after:
        full_commits = list([commits_collection.find_one({'sha': commit_hash, 'date': {'$exists': True}, 'statuses': {'$exists': True}}) for commit_hash in pr["commits"]])
        
        full_commits = [c for c in full_commits if c is not None]
        
        if len(full_commits)== 0:
            continue
            
        full_commits.sort(key = lambda c : c["date"])
        
        statuses = []
        
        for commit in full_commits:
            
            build_res = [status for status in commit["statuses"] if status["target_url"] is not None and status["target_url"].startswith("https://travis-ci.org") and status["state"] != "pending"]
            
            build_res.sort(key = lambda r : r["created_at"])
            
            statuses.extend(build_res)
        
        current_build_status = True
        
        pe_build = 0
            
        for status in statuses:
            new_state = True
            
            if status["state"] == "passed":
                new_state = True
            elif status["state"] == "failure":
                new_state = False
                
            if new_state != current_build_status and new_state:
                pe_build += 1
                
            current_build_status = new_state
            
        pr["pseudo_effective_builds"] = pe_build
        
        pull_requests_collection.update_one({"_id": pr["_id"]}, {"$set": {"pseudo_effective_builds" : pr["pseudo_effective_builds"]}})
            
        